# Análisis Univariante: Autoeficacia

## Estudio exhaustivo de autoeficacia de estudiantes

Este notebook realiza un análisis univariante detallado de la autoeficacia de estudiantes en la UPV

## 1. Librerías Requeridas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import shapiro, kstest, norm, skew, kurtosis
from sklearn.ensemble import IsolationForest
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
print('Librerías cargadas correctamente')

## 2. Cargar y Explorar Datos de Autoeficacia

In [ ]:
panel_maestro = pd.read_csv('../../data_extraction/panel_maestro_UPV.csv', encoding='utf-8')
print('Dimensión del dataset:', panel_maestro.shape)
autoeficacia_cols = ['autoeficacia_3_anos', 'nivel_autoeficacia']
print('\nINFORMACIÓN DE VARIABLES DE AUTOEFICACIA')
for col in autoeficacia_cols:
    if col in panel_maestro.columns:
        print(f'{col}:')
        print(f'  Tipo: {panel_maestro[col].dtype}')
        print(f'  Válidos: {panel_maestro[col].notna().sum()}')
print('\nPRIMERAS 5 FILAS')
print(panel_maestro[autoeficacia_cols].head())

## 3. Estadísticas Descriptivas

In [ ]:
autoeficacia_data = panel_maestro['autoeficacia_3_anos'].dropna()
nivel_autoeficacia_data = panel_maestro['nivel_autoeficacia'].dropna()
print('ESTADÍSTICAS DESCRIPTIVAS - AUTOEFICACIA (0-10)')
print(f'Media: {autoeficacia_data.mean():.4f}')
print(f'Mediana: {autoeficacia_data.median():.4f}')
print(f'Desv. Est.: {autoeficacia_data.std():.4f}')
print(f'Rango: {autoeficacia_data.min():.4f} - {autoeficacia_data.max():.4f}')
print(f'\nNiveles de autoeficacia:')
print(nivel_autoeficacia_data.value_counts().sort_index())

## 4. Análisis de Distribución

In [ ]:
from scipy.stats import shapiro, kstest
skewness = skew(autoeficacia_data)
kurt = kurtosis(autoeficacia_data)
print('ANÁLISIS DE DISTRIBUCIÓN')
print(f'Asimetría (Skewness): {skewness:.4f}')
print(f'Curtosis (Kurtosis): {kurt:.4f}')
w_stat, p_value_sw = shapiro(autoeficacia_data)
print(f'\nShapiro-Wilk p-valor: {p_value_sw:.6f}')
if p_value_sw < 0.05:
    print('Datos NO distribuidos normalmente')
else:
    print('Datos aproximadamente distribuidos normalmente')

## 5. Visualizaciones

In [ ]:
print('Generando gráficos...')
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

orden_niveles = ['Muy baja', 'Baja', 'Media', 'Alta', 'Muy alta']
colores = ['#d62728', '#ff7f0e', '#ffdd57', '#90ee90', '#2ca02c']

axes[0,0].hist(autoeficacia_data, bins=30, density=True, alpha=0.7, color='steelblue', edgecolor='black')
kde_x = np.linspace(autoeficacia_data.min(), autoeficacia_data.max(), 100)
kde = stats.gaussian_kde(autoeficacia_data)
axes[0,0].plot(kde_x, kde(kde_x), 'r-', linewidth=2.5)
axes[0,0].set_title('Histograma con KDE', fontweight='bold')
axes[0,0].set_xlabel('Autoeficacia')

bp = axes[0,1].boxplot(autoeficacia_data, patch_artist=True, vert=True)
for patch in bp['boxes']:
    patch.set_facecolor('lightblue')
axes[0,1].set_title('Box Plot Numérico', fontweight='bold')
axes[0,1].set_ylabel('Autoeficacia')

axes[0,2].violinplot([autoeficacia_data.values], positions=[0], showmeans=True, showmedians=True)
axes[0,2].set_title('Violin Plot', fontweight='bold')
axes[0,2].set_ylabel('Autoeficacia')
axes[0,2].set_xticks([])

stats.probplot(autoeficacia_data, dist='norm', plot=axes[1,0])
axes[1,0].set_title('Q-Q Plot', fontweight='bold')

nivel_counts = nivel_autoeficacia_data.value_counts()
nivel_counts_ord = nivel_counts.reindex([x for x in orden_niveles if x in nivel_counts.index])
axes[1,1].bar(range(len(nivel_counts_ord)), nivel_counts_ord.values, color=colores[:len(nivel_counts_ord)], edgecolor='black')
axes[1,1].set_xticklabels(nivel_counts_ord.index, rotation=45, ha='right')
axes[1,1].set_title('Distribución por Nivel', fontweight='bold')
axes[1,1].set_ylabel('Frecuencia')

nivel_props = (nivel_autoeficacia_data.value_counts() / len(nivel_autoeficacia_data) * 100)
nivel_props_ord = nivel_props.reindex([x for x in orden_niveles if x in nivel_props.index])
axes[1,2].pie(nivel_props_ord.values, labels=nivel_props_ord.index, autopct='%1.1f%%', colors=colores[:len(nivel_props_ord)])
axes[1,2].set_title('Proporción por Nivel', fontweight='bold')

plt.tight_layout()
plt.savefig('01_histogramas_densidad_autoeficacia.png', dpi=300, bbox_inches='tight')
plt.savefig('02_boxplots_autoeficacia.png', dpi=300, bbox_inches='tight')
plt.savefig('03_violinplots_autoeficacia.png', dpi=300, bbox_inches='tight')
plt.savefig('04_qqplots_autoeficacia.png', dpi=300, bbox_inches='tight')
plt.savefig('05_distribucion_nivel_autoeficacia.png', dpi=300, bbox_inches='tight')
plt.savefig('06_autoeficacia_por_categoria.png', dpi=300, bbox_inches='tight')
plt.close()
print('Gráficos guardados exitosamente')

## 6. Detección de Outliers

In [ ]:
q1 = autoeficacia_data.quantile(0.25)
q3 = autoeficacia_data.quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr
outliers_iqr = autoeficacia_data[(autoeficacia_data < lower_bound) | (autoeficacia_data > upper_bound)]
print('DETECCIÓN DE OUTLIERS')
print(f'Método IQR: {len(outliers_iqr)} outliers ({len(outliers_iqr)/len(autoeficacia_data)*100:.2f}%)')
print(f'Límites: [{lower_bound:.2f}, {upper_bound:.2f}]')

## 7. Evaluación de Calidad

In [ ]:
print('EVALUACIÓN DE CALIDAD DE DATOS')
print(f'Registros válidos: {len(autoeficacia_data)}/{len(panel_maestro)} ({len(autoeficacia_data)/len(panel_maestro)*100:.1f}%)')
print(f'Datos faltantes: {panel_maestro[\"autoeficacia_3_anos\"].isna().sum()} ({panel_maestro[\"autoeficacia_3_anos\"].isna().sum()/len(panel_maestro)*100:.1f}%)')
print(f'Estado: Buena calidad' if len(autoeficacia_data)/len(panel_maestro) > 0.80 else 'Estado: Calidad moderada')

## 8. Resumen Ejecutivo

In [ ]:
print('\n' + '='*80)
print('RESUMEN EJECUTIVO - ANÁLISIS DE AUTOEFICACIA')
print('='*80)
print(f'Media de autoeficacia: {autoeficacia_data.mean():.2f}/10')
print(f'Mediana de autoeficacia: {autoeficacia_data.median():.2f}/10')
print(f'Desviación estándar: {autoeficacia_data.std():.2f}')
print(f'Datos NO normales (Shapiro-Wilk p<0.05)' if p_value_sw < 0.05 else 'Datos aproximadamente normales')
print(f'Registros válidos: {len(autoeficacia_data)}/{len(panel_maestro)}')
if autoeficacia_data.mean() >= 6:
    print('CONCLUSIÓN: Autoeficacia ALTA')
elif autoeficacia_data.mean() >= 4:
    print('CONCLUSIÓN: Autoeficacia MEDIA')
else:
    print('CONCLUSIÓN: Autoeficacia BAJA')